# 1) 설정 · 로드 · 공통 유틸

In [ ]:
# -*- coding: utf-8 -*-
# ===============================================================
# 개선 패키지: SKU 개별 & 통합 리드타임, 병목, 생산량 + 탄력도(민감도, 경량)
# - 입력: CSV 경로 (기본: "./Final Results Extended.csv")
# - 산출: ./outputs/ 아래에 모든 결과 저장
# - 주요 개선:
#   1) 생산량 타깃 누수 제거(c_TotalProducts 피처 제외)
#   2) 행별 병목 라벨 생성 → SKU별 리드타임과 조건부 연계표 저장
#   3) 리드타임/생산량 탄력도: (a) 빠른 유한차분법(FD) + 샘플링, (b) (옵션) PDP
#   4) What-if 시뮬레이터 (통합 & SKU별)
#   5) (옵션) 결함(불량) 타깃 자동 탐지 동일 파이프라인
#   6) (옵션) mu/sd 기반 편차 파생피처
# ===============================================================

import os, re, json, math, gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import Dict, List, Tuple, Optional

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge

# ---- 경로/표시 설정 ----
CSV_PATH = "./Final Results Extended.csv"   # 필요시 변경
OUT_DIR  = "./outputs"
os.makedirs(OUT_DIR, exist_ok=True)

pd.set_option("display.max_columns", 160)
pd.set_option("display.width", 160)

# ---- 데이터 로드 ----
df = pd.read_csv(CSV_PATH, low_memory=False)
print("=== Dataset shape ===", df.shape)
display(df.head(3))

# ---- 공통 유틸 ----
def to_numeric_df(d: pd.DataFrame) -> pd.DataFrame:
    return d.apply(pd.to_numeric, errors="coerce").fillna(0.0).astype("float32")

def savefig(path: str):
    plt.tight_layout()
    plt.savefig(path, bbox_inches="tight", dpi=140)
    plt.show()

def metrics_dict(y_true, y_pred) -> dict:
    mae  = float(mean_absolute_error(y_true, y_pred))
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    r2   = float(r2_score(y_true, y_pred))
    return {"MAE": round(mae,3), "RMSE": round(rmse,3), "R2": round(r2,4)}

def train_rf(X: pd.DataFrame, y: np.ndarray, random_state=42, n_estimators=300, test_size=0.3):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    rf = RandomForestRegressor(
        n_estimators=n_estimators, max_depth=None, random_state=random_state, n_jobs=-1
    )
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    m = metrics_dict(y_test, y_pred)
    fi = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
    pi = permutation_importance(rf, X_test, y_test, n_repeats=10, random_state=random_state, n_jobs=-1)
    pi_s = pd.Series(pi.importances_mean, index=X.columns).sort_values(ascending=False)
    return rf, (X_train, X_test, y_train, y_test), m, fi, pi_s

def train_ridge(X: pd.DataFrame, y: np.ndarray, alpha=1.0, random_state=42, test_size=0.3):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    reg = Ridge(alpha=alpha, random_state=random_state)
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    m = metrics_dict(y_test, y_pred)
    coef = pd.Series(reg.coef_, index=X.columns).sort_values(ascending=False)
    std_x = X_test.std(axis=0).replace(0, np.nan)
    std_y = np.std(y_test) if np.std(y_test) > 0 else np.nan
    beta_std = (coef * (std_x / std_y)).sort_values(ascending=False)
    return reg, (X_train, X_test, y_train, y_test), m, coef, beta_std

def topk(s: pd.Series, k=20):
    return s.head(k) if len(s) > k else s

def ensure_dir(path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    return path

# 2) 컬럼 인식 · SKU 타깃 · 통합 리드타임(메모리 안전)

In [ ]:
# === 컬럼 그룹핑 ===
queue_cols = [c for c in df.columns if c.endswith("_Queue")]
util_cols  = [c for c in df.columns if c.endswith("_Util")]
cycle_cols = [c for c in df.columns if c.startswith("c_Cycle")]
cell_cols  = [c for c in df.columns if c.startswith("c_Cell")]

print("\n=== Column groups ===")
print(f"Queue cols  ({len(queue_cols)}): {queue_cols[:10]}{' ...' if len(queue_cols)>10 else ''}")
print(f"Util cols   ({len(util_cols)}):  {util_cols[:10]}{' ...' if len(util_cols)>10 else ''}")
print(f"Cycle cols  ({len(cycle_cols)}): {cycle_cols[:10]}")
print(f"Cell cols   ({len(cell_cols)}):  {cell_cols[:10]}{' ...' if len(cell_cols)>10 else ''}")

# === SKU 자동 탐지 ===
sku_ids = sorted({re.findall(r"SKU(\d+)_", c)[0] for c in df.columns if re.findall(r"SKU(\d+)_", c)})
print("\n=== Detected SKUs ===", sku_ids)

# 리드타임 컬럼 후보
LT_PATTERNS = dict(
    VA=["_VA_Time_sec", "_VA_Time", "_VA"],
    WAIT=["_Wait_Time_sec", "_Wait_Time", "_Wait"],
    LIFT=["_Transport_Time_sec", "_Transport_Time", "_Transport", "_Lift"],
    TOTAL=["_Total_sec", "_LeadTime_sec", "_LeadTime"]
)
def find_first_existing(base: str, suffixes: List[str]) -> Optional[str]:
    for s in suffixes:
        col = base + s
        if col in df.columns:
            return col
    return None

# === SKU별 리드타임/생산량 타깃 ===
sku_targets = {}
for sid in sku_ids:
    base = f"SKU{sid}"
    col_VA   = find_first_existing(base, LT_PATTERNS["VA"])
    col_WAIT = find_first_existing(base, LT_PATTERNS["WAIT"])
    col_LIFT = find_first_existing(base, LT_PATTERNS["LIFT"])
    col_TOT  = find_first_existing(base, LT_PATTERNS["TOTAL"])

    # 리드타임 타깃
    if col_TOT:
        lt_series = pd.to_numeric(df[col_TOT], errors="coerce").fillna(0.0).astype("float32")
        lt_name, lt_method = col_TOT, "TOTAL_col"
    else:
        VA   = pd.to_numeric(df[col_VA],   errors="coerce").fillna(0.0).astype("float32") if col_VA   else 0.0
        WAIT = pd.to_numeric(df[col_WAIT], errors="coerce").fillna(0.0).astype("float32") if col_WAIT else 0.0
        LIFT = pd.to_numeric(df[col_LIFT], errors="coerce").fillna(0.0).astype("float32") if col_LIFT else 0.0
        lt_series = pd.Series(VA, index=df.index) + pd.Series(WAIT, index=df.index) + pd.Series(LIFT, index=df.index)
        lt_series = lt_series.astype("float32")
        lt_name, lt_method = f"{base}_LeadTime_composed", "VA+WAIT+LIFT"

    # 생산량 타깃: 해당 SKU의 셀 출력 합
    cell_cols_sku = [c for c in cell_cols if re.search(fr"__SKU{sid}\b", c)]
    if cell_cols_sku:
        prod_series = to_numeric_df(df[cell_cols_sku]).sum(axis=1).astype("float32")
        prod_name   = f"Total_SKU{sid}"
    else:
        prod_series = pd.Series(0.0, index=df.index, dtype="float32")
        prod_name   = f"Total_SKU{sid}_zeros"

    sku_targets[sid] = dict(
        lead_time=lt_series, lead_time_name=lt_name, lead_time_method=lt_method,
        production=prod_series, production_name=prod_name, cell_cols=cell_cols_sku
    )

print("\n=== SKU targets overview (first 2) ===")
for sid in sku_ids[:2]:
    print(f"SKU{sid} LT: {sku_targets[sid]['lead_time_name']} ({sku_targets[sid]['lead_time_method']}), "
          f"PROD: {sku_targets[sid]['production_name']} | #cell_cols={len(sku_targets[sid]['cell_cols'])}")

# === 통합 타깃 ===
# 생산량(통합)
if "c_TotalProducts" in df.columns:
    total_prod = pd.to_numeric(df["c_TotalProducts"], errors="coerce").fillna(0.0).astype("float32")
else:
    total_prod = to_numeric_df(df[cell_cols]).sum(axis=1).astype("float32")

# 리드타임(생산량 가중평균, 메모리 안전)
N = len(df)
numerator = np.zeros(N, dtype=np.float32)  # Σ(lt_k * prod_k)
weights   = np.zeros(N, dtype=np.float32)  # Σ(prod_k)
sum_lt    = np.zeros(N, dtype=np.float32)  # fallback 평균 분자
k_count   = 0

for sid in sku_ids:
    lt = sku_targets[sid]["lead_time"].values
    pr = sku_targets[sid]["production"].values
    numerator += lt * pr
    weights   += pr
    sum_lt    += lt
    k_count   += 1

fallback_mean = (sum_lt / max(k_count, 1)).astype("float32")
with np.errstate(divide='ignore', invalid='ignore'):
    weighted_lt = np.where(weights > 0.0, numerator / weights, fallback_mean)
agg_lead_time = pd.Series(weighted_lt, index=df.index, name="Agg_LeadTime_weighted").astype("float32")

del numerator, weights, sum_lt; gc.collect()

# 3) 병목 라벨링 · 요약표(전체/통합) · SKU×병목 연계표

In [ ]:
# === 행별 병목 라벨링 ===
Q = to_numeric_df(df[[c for c in df.columns if c.endswith("_Queue")]])
if Q.shape[1] == 0:
    raise ValueError("Queue 계열 컬럼이 없습니다.")
df["_bneck"] = Q.idxmax(axis=1)  # 각 행 최대 Queue의 컬럼명

def compute_bneck_stats(target: pd.Series, name: str, out_csv: str):
    tmp = pd.DataFrame({"bneck": df["_bneck"].astype("category"), "val": target.astype("float32")})
    stats = tmp.groupby("bneck").agg(
        top1_count=("bneck", "count"),
        mean_target=("val", "mean"),
        std_target=("val", "std")
    ).sort_values("top1_count", ascending=False).reset_index()
    stats["target_name"] = name
    stats.to_csv(ensure_dir(os.path.join(OUT_DIR, out_csv)), index=False)
    return stats

bneck_total_prod = compute_bneck_stats(total_prod, "TotalProduction", "bottleneck_totalProduction.csv")
bneck_agg_lt     = compute_bneck_stats(agg_lead_time, "AggLeadTime", "bottleneck_aggLeadTime.csv")

display(bneck_total_prod.head(10))
display(bneck_agg_lt.head(10))

# === SKU별: 병목 ↔ 리드타임 조건부 연계표 ===
TOP_BNECKS = list(bneck_total_prod["bneck"].head(8).values)  # 빈도 상위 N
for sid in sku_ids:
    lt = sku_targets[sid]["lead_time"].astype("float32")
    tmp = pd.DataFrame({"bneck": df["_bneck"], "lt": lt})
    tmp["bneck_top"] = np.where(tmp["bneck"].isin(TOP_BNECKS), tmp["bneck"], "Other")
    grp = tmp.groupby("bneck_top")["lt"].agg(["count","mean","std"]).sort_values("count", ascending=False)
    grp.to_csv(ensure_dir(os.path.join(OUT_DIR, f"sku_bneck_effect_SKU{sid}.csv")))

# 4) 피처 구성(누수 방지 + 병목 One-Hot) · 모델 학습(RF & Ridge)

In [ ]:
# === 피처 구성 ===
time_like_regex = re.compile(r"(?:_Time|_sec|_LeadTime|_Total)", re.IGNORECASE)

def build_features_for_leadtime(df_: pd.DataFrame, exclude_cols: List[str],
                                add_bneck_onehot=True, top_bneck_list=None) -> pd.DataFrame:
    cols = []
    for c in df_.columns:
        if c in exclude_cols or c == "Time_Now":
            continue
        if time_like_regex.search(c):      # 시간/리드타임 직접 계열 제외(누수 방지)
            continue
        cols.append(c)
    X = to_numeric_df(df_[cols]) if cols else pd.DataFrame(index=df_.index)
    if add_bneck_onehot and "_bneck" in df_.columns:
        b = df_["_bneck"].astype(str)
        if top_bneck_list is not None:
            b = np.where(b.isin(top_bneck_list), b, "Other")
            b = pd.Series(b, index=df_.index)
        D = pd.get_dummies(b, prefix="BNECK", dtype="float32")
        X = pd.concat([X, D], axis=1)
    return X

def build_features_for_production(df_: pd.DataFrame, exclude_cols: List[str]) -> pd.DataFrame:
    cols = []
    for c in df_.columns:
        if c in exclude_cols or c == "Time_Now":
            continue
        if c.startswith("c_Cell"):         # 직접 합산되는 생산량 열 제외
            continue
        if c == "c_TotalProducts":         # 통합 생산량 타깃 누수 제거
            continue
        cols.append(c)
    return to_numeric_df(df_[cols]) if cols else pd.DataFrame(index=df_.index)

# === 통합(AGG) 모델 ===
# LeadTime(AGG)
X_lt_agg = build_features_for_leadtime(df, exclude_cols=[], add_bneck_onehot=True, top_bneck_list=TOP_BNECKS)
y_lt_agg = agg_lead_time.values
rf_lt_agg, splits_lt_agg, m_rf_lt_agg, fi_lt_agg, pi_lt_agg = train_rf(X_lt_agg, y_lt_agg)
rg_lt_agg, splits_rg_lt_agg, m_rg_lt_agg, coef_lt_agg, beta_std_lt_agg = train_ridge(X_lt_agg, y_lt_agg)

fi_lt_agg.to_csv(ensure_dir(os.path.join(OUT_DIR, "fi_leadtime_AGG.csv")))
pi_lt_agg.to_csv(ensure_dir(os.path.join(OUT_DIR, "pi_leadtime_AGG.csv")))
coef_lt_agg.to_csv(ensure_dir(os.path.join(OUT_DIR, "ridge_coef_leadtime_AGG.csv")))
beta_std_lt_agg.to_csv(ensure_dir(os.path.join(OUT_DIR, "ridge_betaStd_leadtime_AGG.csv")))
plt.figure(figsize=(8,6)); topk(fi_lt_agg,20).iloc[::-1].plot(kind="barh"); plt.title("RF FI (LeadTime) - AGG"); savefig(os.path.join(OUT_DIR,"fi_leadtime_AGG_top20.png"))

# Production(AGG)
X_pr_agg = build_features_for_production(df, exclude_cols=[])
y_pr_agg = total_prod.values
rf_pr_agg, splits_pr_agg, m_rf_pr_agg, fi_pr_agg, pi_pr_agg = train_rf(X_pr_agg, y_pr_agg)
rg_pr_agg, splits_rg_pr_agg, m_rg_pr_agg, coef_pr_agg, beta_std_pr_agg = train_ridge(X_pr_agg, y_pr_agg)

fi_pr_agg.to_csv(ensure_dir(os.path.join(OUT_DIR, "fi_production_AGG.csv")))
pi_pr_agg.to_csv(ensure_dir(os.path.join(OUT_DIR, "pi_production_AGG.csv")))
coef_pr_agg.to_csv(ensure_dir(os.path.join(OUT_DIR, "ridge_coef_production_AGG.csv")))
beta_std_pr_agg.to_csv(ensure_dir(os.path.join(OUT_DIR, "ridge_betaStd_production_AGG.csv")))
plt.figure(figsize=(8,6)); topk(fi_pr_agg,20).iloc[::-1].plot(kind="barh"); plt.title("RF FI (Production) - AGG"); savefig(os.path.join(OUT_DIR,"fi_production_AGG_top20.png"))

print("\n=== AGG Metrics ===")
print({"RF_LT": m_rf_lt_agg, "RG_LT": m_rg_lt_agg, "RF_PR": m_rf_pr_agg, "RG_PR": m_rg_pr_agg})

# === SKU별 모델 ===
per_sku_results = {}
for sid in sku_ids:
    # LeadTime
    y_lt = sku_targets[sid]["lead_time"].values
    exclude_lt = set(sku_targets[sid]["cell_cols"])
    exclude_lt |= {c for c in df.columns if c.startswith(f"SKU{sid}_") and time_like_regex.search(c)}
    X_lt = build_features_for_leadtime(df, list(exclude_lt), add_bneck_onehot=True, top_bneck_list=TOP_BNECKS)

    if X_lt.shape[1] > 0:
        rf_lt, spl_lt, m_rf_lt, fi_lt, pi_lt = train_rf(X_lt, y_lt)
        rg_lt, spl_rg_lt, m_rg_lt, coef_lt, beta_std_lt = train_ridge(X_lt, y_lt)
        fi_lt.to_csv(ensure_dir(os.path.join(OUT_DIR, f"fi_leadtime_SKU{sid}.csv")))
        pi_lt.to_csv(ensure_dir(os.path.join(OUT_DIR, f"pi_leadtime_SKU{sid}.csv")))
        coef_lt.to_csv(ensure_dir(os.path.join(OUT_DIR, f"ridge_coef_leadtime_SKU{sid}.csv")))
        beta_std_lt.to_csv(ensure_dir(os.path.join(OUT_DIR, f"ridge_betaStd_leadtime_SKU{sid}.csv")))
        plt.figure(figsize=(8,6)); topk(fi_lt,20).iloc[::-1].plot(kind="barh"); plt.title(f"RF FI (LeadTime) - SKU{sid}"); savefig(os.path.join(OUT_DIR, f"fi_leadtime_SKU{sid}_top20.png"))
        plt.figure(figsize=(8,6)); topk(pi_lt,20).iloc[::-1].plot(kind="barh"); plt.title(f"Permutation FI (LeadTime) - SKU{sid}"); savefig(os.path.join(OUT_DIR, f"pi_leadtime_SKU{sid}_top20.png"))
        lt_pack = dict(rf=(rf_lt, spl_lt, m_rf_lt), rg=(rg_lt, spl_rg_lt, m_rg_lt),
                       fi=fi_lt, pi=pi_lt, coef=coef_lt, beta_std=beta_std_lt, feats=list(X_lt.columns))
    else:
        print(f"[WARN] SKU{sid}: LeadTime features empty → skip")
        lt_pack = None

    # Production
    y_pr = sku_targets[sid]["production"].values
    exclude_pr = set(sku_targets[sid]["cell_cols"])
    X_pr = build_features_for_production(df, list(exclude_pr))

    if X_pr.shape[1] > 0:
        rf_pr, spl_pr, m_rf_pr, fi_pr, pi_pr = train_rf(X_pr, y_pr)
        rg_pr, spl_rg_pr, m_rg_pr, coef_pr, beta_std_pr = train_ridge(X_pr, y_pr)
        fi_pr.to_csv(ensure_dir(os.path.join(OUT_DIR, f"fi_production_SKU{sid}.csv")))
        pi_pr.to_csv(ensure_dir(os.path.join(OUT_DIR, f"pi_production_SKU{sid}.csv")))
        coef_pr.to_csv(ensure_dir(os.path.join(OUT_DIR, f"ridge_coef_production_SKU{sid}.csv")))
        beta_std_pr.to_csv(ensure_dir(os.path.join(OUT_DIR, f"ridge_betaStd_production_SKU{sid}.csv")))
        plt.figure(figsize=(8,6)); topk(fi_pr,20).iloc[::-1].plot(kind="barh"); plt.title(f"RF FI (Production) - SKU{sid}"); savefig(os.path.join(OUT_DIR, f"fi_production_SKU{sid}_top20.png"))
        plt.figure(figsize=(8,6)); topk(pi_pr,20).iloc[::-1].plot(kind="barh"); plt.title(f"Permutation FI (Production) - SKU{sid}"); savefig(os.path.join(OUT_DIR, f"pi_production_SKU{sid}_top20.png"))
        pr_pack = dict(rf=(rf_pr, spl_pr, m_rf_pr), rg=(rg_pr, spl_rg_pr, m_rg_pr),
                       fi=fi_pr, pi=pi_pr, coef=coef_pr, beta_std=beta_std_pr, feats=list(X_pr.columns))
    else:
        print(f"[WARN] SKU{sid}: Production features empty → skip")
        pr_pack = None

    per_sku_results[sid] = dict(lead_time=lt_pack, production=pr_pack)

# 5) 탄력도(민감도) 정량화: PDP 기울기 + 선형계수

In [ ]:
# === 경량 탄력도 설정 ===
from numpy.random import default_rng
RANDOM_STATE = 42
_rng = default_rng(RANDOM_STATE)

# 중요도 기반 후보 피처 선택 (Queue/Util 우선)
FEATURES_TOP_K = 12          # 너무 크면 시간 증가
ONLY_QUEUE_UTIL = True
SAMPLE_N = 20000             # 2~5만 추천
USE_FDP_FAST = True          # 빠른 유한차분법(권장). False면 PDP(느림)

def _pick_features(X: pd.DataFrame, fi: Optional[pd.Series]) -> List[str]:
    cols = X.columns.tolist()
    if ONLY_QUEUE_UTIL:
        base = [c for c in cols if c.endswith("_Queue")] + [c for c in cols if c.endswith("_Util")]
        if fi is None:
            return base[:FEATURES_TOP_K] if base else cols[:FEATURES_TOP_K]
        ranked = [c for c in fi.index if c in base]
        if not ranked:
            ranked = [c for c in fi.index if c in cols]
        return ranked[:FEATURES_TOP_K]
    else:
        if fi is None:
            return cols[:FEATURES_TOP_K]
        return [c for c in fi.index if c in cols][:FEATURES_TOP_K]

ELASTICITY_FEATURES_LT = _pick_features(X_lt_agg, fi_lt_agg)
ELASTICITY_FEATURES_PR = _pick_features(X_pr_agg, fi_pr_agg)
print("ELASTICITY_FEATURES_LT:", ELASTICITY_FEATURES_LT)
print("ELASTICITY_FEATURES_PR:", ELASTICITY_FEATURES_PR)

# --- 빠른 유한차분법(FD) ---
def _suggest_h(x: np.ndarray) -> float:
    if x.size == 0: return 1.0
    q25, q75 = np.percentile(x, [25, 75])
    iqr = q75 - q25
    if iqr > 0: return float(iqr * 0.05)
    std = np.std(x)
    return float(std * 0.1 if std > 0 else 1.0)

def fd_slope_mean(model, X: pd.DataFrame, feature: str, h: Optional[float]=None, sample_n: int=20000) -> Optional[float]:
    if feature not in X.columns: return None
    n = len(X)
    if n == 0: return None
    if sample_n and sample_n < n:
        idx = _rng.choice(n, size=sample_n, replace=False)
        Xs = X.iloc[idx].copy()
    else:
        Xs = X.copy()
    x = Xs[feature].to_numpy()
    if h is None: h = _suggest_h(x)
    if h == 0: return 0.0
    X_minus = Xs.copy(); X_plus = Xs.copy()
    X_minus[feature] = x - h
    X_plus[feature]  = x + h
    y_m = model.predict(X_minus).mean()
    y_p = model.predict(X_plus).mean()
    return float((y_p - y_m) / (2.0 * h))

# --- (옵션) PDP 기반 — 정확하지만 느림 ---
from sklearn.inspection import partial_dependence
def pdp_slope_mean(model, X: pd.DataFrame, feature: str, h: Optional[float]=None, sample_n: int=20000) -> Optional[float]:
    if feature not in X.columns: return None
    n = len(X)
    if n == 0: return None
    if sample_n and sample_n < n:
        idx = _rng.choice(n, size=sample_n, replace=False)
        Xs = X.iloc[idx].copy()
    else:
        Xs = X.copy()
    x = Xs[feature].to_numpy()
    if h is None: h = _suggest_h(x)
    grid = [np.median(x) - h, np.median(x), np.median(x) + h]
    pdp = partial_dependence(model, Xs, [feature], grid=grid, kind="average")
    ys = pdp.average[0]
    xs = np.array(grid, dtype=float)
    denom = (xs[-1] - xs[0])
    return float((ys[-1] - ys[0]) / denom) if denom != 0 else 0.0

def run_elasticity_block(tag: str, model, X: pd.DataFrame, features: List[str],
                         use_fast: bool=True, sample_n: int=20000) -> pd.DataFrame:
    rows = []
    for f in features:
        s = fd_slope_mean(model, X, f, h=None, sample_n=sample_n) if use_fast else pdp_slope_mean(model, X, f, h=None, sample_n=sample_n)
        rows.append({"feature": f, "slope": s})
    out = pd.DataFrame(rows).dropna().sort_values("slope", ascending=False)
    out.to_csv(os.path.join(OUT_DIR, f"elasticity_{tag}.csv"), index=False)
    return out

elasticity_outputs = {}

# AGG LeadTime
el_lt = run_elasticity_block(
    tag="leadtime_AGG_fast" if USE_FDP_FAST else "leadtime_AGG_pdp",
    model=rf_lt_agg, X=X_lt_agg, features=ELASTICITY_FEATURES_LT,
    use_fast=USE_FDP_FAST, sample_n=SAMPLE_N
)
elasticity_outputs["leadtime_AGG"] = el_lt
display(el_lt.head(15))

# AGG Production
el_pr = run_elasticity_block(
    tag="production_AGG_fast" if USE_FDP_FAST else "production_AGG_pdp",
    model=rf_pr_agg, X=X_pr_agg, features=ELASTICITY_FEATURES_PR,
    use_fast=USE_FDP_FAST, sample_n=SAMPLE_N
)
elasticity_outputs["production_AGG"] = el_pr
display(el_pr.head(15))

# (선택) SKU별에도 동일 적용
per_sku_elasticity = {}
for sid, packs in per_sku_results.items():
    per_sku_elasticity[sid] = {}
    # LeadTime
    if packs.get("lead_time"):
        rf = packs["lead_time"]["rf"][0]
        Xte = packs["lead_time"]["rf"][1][1]
        feats = [f for f in ELASTICITY_FEATURES_LT if f in packs["lead_time"]["feats"]]
        if feats:
            tag = f"leadtime_SKU{sid}_" + ("fast" if USE_FDP_FAST else "pdp")
            out = run_elasticity_block(tag, rf, Xte, feats, USE_FDP_FAST, SAMPLE_N)
            per_sku_elasticity[sid]["lead_time"] = out
    # Production
    if packs.get("production"):
        rf = packs["production"]["rf"][0]
        Xte = packs["production"]["rf"][1][1]
        feats = [f for f in ELASTICITY_FEATURES_PR if f in packs["production"]["feats"]]
        if feats:
            tag = f"production_SKU{sid}_" + ("fast" if USE_FDP_FAST else "pdp")
            out = run_elasticity_block(tag, rf, Xte, feats, USE_FDP_FAST, SAMPLE_N)
            per_sku_elasticity[sid]["production"] = out

print("Elasticity CSVs saved to:", OUT_DIR)

# 6) What‑if 시뮬레이터 (AGG & SKU별)

In [ ]:
def simulate_delta(model, X_base: pd.DataFrame, deltas: Dict[str, float]) -> Tuple[float, float, float]:
    X_sim = X_base.copy()
    for k, v in deltas.items():
        if k in X_sim.columns:
            X_sim[k] = X_sim[k] + v
    base_mean = float(model.predict(X_base).mean())
    new_mean  = float(model.predict(X_sim).mean())
    return base_mean, new_mean, new_mean - base_mean

SCENARIOS = {
    "BlankingQ_-50": {"Blanking_Queue": -50.0} if "Blanking_Queue" in X_lt_agg.columns else {},
    "Warehouse1Q_-50": {"Warehouse1_Queue": -50.0} if "Warehouse1_Queue" in X_lt_agg.columns else {},
    "QualityUtil_+5": {"Quality_Util": +5.0} if "Quality_Util" in X_lt_agg.columns else {},
}

whatif = {"AGG": {"lead_time": {}, "production": {}}, "SKU": {}}

# AGG LT
Xb_lt = splits_lt_agg[1]
for name, d in SCENARIOS.items():
    if not d: continue
    base_m, new_m, d_m = simulate_delta(rf_lt_agg, Xb_lt, d)
    whatif["AGG"]["lead_time"][name] = {"base_mean": base_m, "new_mean": new_m, "delta_mean": d_m}

# AGG Production
Xb_pr = splits_pr_agg[1]
for name, d in SCENARIOS.items():
    if not d: continue
    base_m, new_m, d_m = simulate_delta(rf_pr_agg, Xb_pr, d)
    whatif["AGG"]["production"][name] = {"base_mean": base_m, "new_mean": new_m, "delta_mean": d_m}

# SKU별
for sid, packs in per_sku_results.items():
    whatif["SKU"][sid] = {}
    if packs["lead_time"] is not None:
        rf, (Xtr, Xte, ytr, yte), _ = packs["lead_time"]["rf"]
        feats = packs["lead_time"]["feats"]
        lt_map = {}
        for name, d in SCENARIOS.items():
            d_sub = {k:v for k,v in d.items() if k in feats}
            if not d_sub: continue
            base_m, new_m, d_m = simulate_delta(rf, Xte[feats], d_sub)
            lt_map[name] = {"base_mean": base_m, "new_mean": new_m, "delta_mean": d_m}
        whatif["SKU"][sid]["lead_time"] = lt_map
    if packs["production"] is not None:
        rf, (Xtr, Xte, ytr, yte), _ = packs["production"]["rf"]
        feats = packs["production"]["feats"]
        pr_map = {}
        for name, d in SCENARIOS.items():
            d_sub = {k:v for k,v in d.items() if k in feats}
            if not d_sub: continue
            base_m, new_m, d_m = simulate_delta(rf, Xte[feats], d_sub)
            pr_map[name] = {"base_mean": base_m, "new_mean": new_m, "delta_mean": d_m}
        whatif["SKU"][sid]["production"] = pr_map

with open(ensure_dir(os.path.join(OUT_DIR, "whatif_results.json")), "w", encoding="utf-8") as f:
    json.dump(whatif, f, ensure_ascii=False, indent=2)

print("=== What-if (AGG) ===")
print(json.dumps(whatif["AGG"], ensure_ascii=False, indent=2))

# 7) (옵션) 결함/불량 타깃 자동 탐지 · 동일 파이프라인

In [ ]:
DEFECT_PAT = re.compile(r"(defect|scrap|reject|ng|fail)", re.IGNORECASE)
defect_targets = [c for c in df.columns if DEFECT_PAT.search(c)]

if defect_targets:
    print("=== Detected defect targets ===", defect_targets)
    for tgt in defect_targets:
        y_df = pd.to_numeric(df[tgt], errors="coerce").fillna(0.0).astype("float32").values
        X_df = build_features_for_leadtime(df, exclude_cols=[], add_bneck_onehot=True, top_bneck_list=TOP_BNECKS)
        rf, spl, m_rf, fi, pi = train_rf(X_df, y_df)
        rg, spl2, m_rg, coef, beta_std = train_ridge(X_df, y_df)
        base = tgt.replace(os.sep, "_")
        fi.to_csv(ensure_dir(os.path.join(OUT_DIR, f"fi_defect_{base}.csv")))
        pi.to_csv(ensure_dir(os.path.join(OUT_DIR, f"pi_defect_{base}.csv")))
        coef.to_csv(ensure_dir(os.path.join(OUT_DIR, f"ridge_coef_defect_{base}.csv")))
        beta_std.to_csv(ensure_dir(os.path.join(OUT_DIR, f"ridge_betaStd_defect_{base}.csv")))
        print(f"[{tgt}] RF={m_rf}, Ridge={m_rg}")
else:
    print("No defect/scrap targets detected.")

# 8) (옵션) mu/sd 기반 편차 파생피처 추가 – 필요 시만 ON

In [ ]:
INCLUDE_MUSD_IN_LT = False   # 리드타임 모델 포함 여부
INCLUDE_MUSD_IN_PR = True    # 생산량 모델 포함 여부
MUSD: Dict[str, Dict[str, Dict[str, float]]] = {
    "SKU1": {
        "mu": {"BL":900.0, "PR":5.0, "AS":25.0, "PA":5400.0, "QL":55.0},
        "sd": {"BL":30.0,  "PR":0.1, "AS":0.1,  "PA":0.0,    "QL":2.04},
    }
}
REPLICATE_TO_OTHERS = True

def add_musd_features(df_: pd.DataFrame, sku_ids_: List[str]) -> List[str]:
    created = []
    for sid in sku_ids_:
        key = f"SKU{sid}"
        if key not in MUSD and REPLICATE_TO_OTHERS and "SKU1" in MUSD:
            MUSD[key] = MUSD["SKU1"]
        if key not in MUSD:
            continue
        mu = MUSD[key]["mu"]; sd = MUSD[key]["sd"]
        mu_sum = mu["BL"] + mu["PR"] + mu["AS"] + mu["PA"] + mu["QL"]
        S_var  = sd["BL"]**2 + sd["PR"]**2 + sd["AS"]**2 + sd["QL"]**2
        S_std  = math.sqrt(S_var) if S_var > 0 else 1.0
        base = f"SKU{sid}"
        va_col = next((base+s for s in LT_PATTERNS["VA"] if (base+s) in df_.columns), None)
        VA = pd.to_numeric(df_[va_col], errors="coerce").fillna(0.0) if va_col else pd.Series(0.0, index=df_.index)
        dev_col = f"{base}_VA_dev"; z_col = f"{base}_VA_z"
        df_[dev_col] = (VA - mu_sum).astype("float32")
        df_[z_col]   = ((VA - mu_sum)/S_std).astype("float32")
        created += [dev_col, z_col]
    return created

# 사용하려면:
# musd_cols = add_musd_features(df, sku_ids)
# print("MUSD features created:", musd_cols)
# -> 이후 '셀 4' 모델 재학습 시 생산량 모델에는 포함(리드타임 모델은 기본 제외)하도록 응용 가능.

# 9) 요약 리포트 저장

In [ ]:
summary = {
    "cwd": os.getcwd(),
    "dataset_shape": tuple(df.shape),
    "detected_skus": sku_ids,
    "top_bottlenecks_used": TOP_BNECKS,
    "metrics": {
        "AGG": {
            "RF_LT": m_rf_lt_agg, "RG_LT": m_rg_lt_agg,
            "RF_PR": m_rf_pr_agg, "RG_PR": m_rg_pr_agg,
        }
    },
    "outputs": {
        "bottleneck_total": os.path.join(OUT_DIR, "bottleneck_totalProduction.csv"),
        "bottleneck_aggLT": os.path.join(OUT_DIR, "bottleneck_aggLeadTime.csv"),
        "sku_bneck_effect": "sku_bneck_effect_SKU*.csv",
        "agg_fi_pi_lt": ["fi_leadtime_AGG.csv","pi_leadtime_AGG.csv","fi_leadtime_AGG_top20.png",
                         "ridge_coef_leadtime_AGG.csv","ridge_betaStd_leadtime_AGG.csv",
                         "elasticity_leadtime_AGG_fast.csv"],
        "agg_fi_pi_pr": ["fi_production_AGG.csv","pi_production_AGG.csv","fi_production_AGG_top20.png",
                         "ridge_coef_production_AGG.csv","ridge_betaStd_production_AGG.csv",
                         "elasticity_production_AGG_fast.csv"],
        "per_sku_elasticity": "elasticity_(leadtime|production)_SKU*_fast.csv",
        "whatif": os.path.join(OUT_DIR, "whatif_results.json")
    },
    "notes": [
        "생산량 모델에서 c_TotalProducts, c_Cell* 피처 제외 → 타깃 누수 방지.",
        "리드타임 모델에서 *_Time/_sec/_LeadTime/_Total 피처 제외 → 누수 방지.",
        "병목 라벨(_bneck) 원-핫 추가로 병목 상태의 평균적 영향 반영.",
        "탄력도: FD(샘플링)으로 빠르게 ∂ŷ/∂x 추정, 필요시 PDP 대체 가능.",
    ]
}
with open(ensure_dir(os.path.join(OUT_DIR, "SUMMARY.json")), "w", encoding="utf-8") as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)

print("\n=== SUMMARY ===")
print(json.dumps(summary, ensure_ascii=False, indent=2))